In [ ]:
import argparse
from argparse import ArgumentParser
import glob
import cv2
import re
import os, glob, datetime
import numpy as np
import tensorflow as tf
import numpy as np
import time
import math
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract, Reshape
from keras.models import Model, load_model
from tensorflow.python.keras.utils import conv_utils
from keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import keras.backend as K

import skimage
from skimage.metrics import structural_similarity as ssim
from skimage.io import imread, imsave

#from skimage.measure import compare_psnr, compare_ssim




##--------------------------------------------------------------------------------------------------------
##--------------------------------------------------------------------------------------------------------


#########--------------           IMPORTANT NOTE       ----------------###############

# for using pretrained model with sigma = 10, 15, 25, 50, 75, 100
# Please modify the 'args.sigma' by the respective sigma vamue
# Also change the 'args.model_dir' and the 'args.model_name' as defined below.

# Select the path of the directory properly to give access to the models, datasets and results
directory_path =  '_________________________' #'Choose your directory



parser = argparse.ArgumentParser()
# choose if needed
parser.add_argument('--model', default='DIVA', type=str, help='choose a type of model')

parser.add_argument('--set_dir', default=os.path.join(directory_path ,'data/'), type=str, help='directory of test dataset')
parser.add_argument('--set_names', default=['Set12'], type=list, help='name of test dataset')

parser.add_argument('--sigma', default=25, type=int, help='noise level - Choose a sigma value from 10, 15, 25, 50, 75, 100')
parser.add_argument('--model_dir', default=os.path.join(directory_path ,'models/DIVA_models_sigma_10_to_100'), type=str, help='directory of the model')
                                                     
parser.add_argument('--model_name', default='model_sigma10.hdf5', type=str, help='the model name')
parser.add_argument('--result_dir', default=os.path.join(directory_path,'results'), type=str, help='directory of results')
parser.add_argument('--save_result', default=1, type=int, help='save the denoised image, 1 or 0')

parser.add_argument('-f', '--file', required=False)
args = parser.parse_args()


#########--------------           IMPORTANT NOTE       ----------------###############

# for using pretrained model with sigma = 10, 15, 25, 50, 75, 100
# Please modify the 'args.sigma' by the respective sigma vamue
# Also change the 'args.model_dir' and the 'args.model_name' as defined below.

#args.sigma = 15 # '_____' #'Choose a sigma value from 10, 15, 25, 50, 75, 100'
args.model_dir = directory_path +'/models/DIVA_models_sigma_10_to_100'
model = 'model_sigma'+str(args.sigma)+'.hdf5'
args.model_name = model

print(args.sigma)
print(args.model_name)

# Also set 'use_model = False' (which load pretrained models)
use_model = False


##--------------------------------------------------------------------------------------------------------
##--------------------------------------------------------------------------------------------------------




def to_tensor(img):
    if img.ndim == 2:
        return img[np.newaxis,...,np.newaxis]
    elif img.ndim == 3:
        return np.moveaxis(img,2,0)[...,np.newaxis]

def from_tensor(img):
    return np.squeeze(np.moveaxis(img[...,0],0,-1))

def log(*args,**kwargs):
     print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"),*args,**kwargs)

def save_result(result,path):
    path = path if path.find('.') != -1 else path+'.png'
    ext = os.path.splitext(path)[-1]
    if ext in ('.txt','.dlm'):
        np.savetxt(path,result,fmt='%2.4f')
    else:
        imsave(path,np.clip(result,0,1))


def show(x,title=None,cbar=False,figsize=None):
    import matplotlib.pyplot as plt
    plt.figure(figsize=figsize)
    plt.imshow(x,interpolation='nearest',cmap='gray')
    if title:
        plt.title(title)
    if cbar:
        plt.colorbar()
    plt.show()

def psnr(target, ref):
    # Assume target is RGB/BGR image
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = np.sqrt(np.mean(diff ** 2.))
    
    return 20 * np.log10(1. / rmse)


def snr(target, ref):
    # Assume target is RGB/BGR image
    target_data = target.astype(np.float32)
    ref_data = ref.astype(np.float32)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    target_data = target_data.flatten('C')
    
    rmse_diff = np.sqrt(np.mean(diff ** 2.))
    rmse_target = np.sqrt(np.mean(target_data ** 2.))

    return 20 * np.log10(rmse_target / rmse_diff)


def psnr_pois(target, ref):
    # Assume target is RGB/BGR image
    target_data = target.astype(np.float32)
    ref_data = ref#.astype(np.float32)
    
    diff = ref_data - target_data
    diff = diff.flatten('C')
    
    rmse = np.sqrt(np.mean(diff ** 2.))
    
    return 20 * np.log10(1. / rmse)


##----------------------------------------------------------------------------------------------------------------
## -------------------------------------------------------------------------------------------------------------
## -------------------------------------------------------------------------------------------------------------


if __name__ == '__main__':    
    
    if  use_model:
        #choose model depth and filters number
        model = DIVA(depth=10,filters=64,image_channels=1,use_bnorm=True)
        model.load_weights(os.path.join(args.model_dir, args.model_name))
        log('load trained model architecture')
    else:
        print('Model- ',args.model)
        model = load_model(os.path.join(args.model_dir, args.model_name),compile=False)
        log('load trained model')

    if not os.path.exists(args.result_dir):
        os.mkdir(args.result_dir)


#-----------------------------------------------------------------------------------------------------------------------------------
#-----------------------------------------------------------------------------------------------------------------------------------

    for set_cur in args.set_names:  
        
        if not os.path.exists(os.path.join(args.result_dir,set_cur)):
            os.mkdir(os.path.join(args.result_dir,set_cur))
        snrs_in = []
        psnrs_in = []
        ssims_in = [] 
        psnrs = []
        ssims = [] 
        
        for im in os.listdir(os.path.join(args.set_dir,set_cur)): 
            if im.endswith(".jpg") or im.endswith(".bmp") or im.endswith(".png"):
                x = np.array(imread(os.path.join(args.set_dir,set_cur,im)), dtype=np.float32) / 255.0
              #  np.random.seed(seed=0) # for reproducibility
                
                # Gaussian noise case
                y = x + np.random.normal(0, args.sigma/255.0, x.shape) # Add Gaussian noise without clipping
                noise_type = 'Gaus'
                y = y.astype(np.float32)

                y_  = to_tensor(y)
                start_time = time.time()
                x_ = model.predict(y_) # inference
                elapsed_time = time.time() - start_time
                print('%10s : %10s : %2.4f second'%(set_cur,im,elapsed_time))

                x_=from_tensor(x_)

                # calculate for Gaussian noise
                snr_y = snr(x, y)      # input SNR
                psnr_y = psnr(x, y)      # input PSNR
                ssim_y = ssim(x, y)      # input SSIM
                psnr_x_ = psnr(x, x_)
                ssim_x_ = ssim(x, x_)
                print('%10s : %2.4f : %1.4f'%(set_cur,psnr_x_, ssim_x_))



                if args.save_result:
                    name, ext = os.path.splitext(im)
                    show(np.hstack((y,x_))) # show the image
                    save_result(y,path=os.path.join(args.result_dir,set_cur,name+'_'+noise_type+'_noisy_'+str(args.sigma)+ext)) # save the noisy image
                    save_result(x_,path=os.path.join(args.result_dir,set_cur,name+'_'+noise_type+'_denoised_DIVA_'+str(args.sigma)+ext)) # save the denoised image
                
                snrs_in.append(snr_y)     # input SNR
                psnrs_in.append(psnr_y)   # input PSNR
                ssims_in.append(ssim_y)   # input SSIM
                psnrs.append(psnr_x_)
                ssims.append(ssim_x_)
                

        snr_in_avg = np.mean(snrs_in)     # average input SNR
        psnr_in_avg = np.mean(psnrs_in)   # average input PSNR
        ssim_in_avg = np.mean(ssims_in)   # average input SSIM
        psnr_avg = np.mean(psnrs)
        ssim_avg = np.mean(ssims)

        snrs_in.append(snr_in_avg)      # input SNR
        psnrs_in.append(psnr_in_avg)    # input PSNR
        ssims_in.append(ssim_in_avg)    # input SSIM
        psnrs.append(psnr_avg)
        ssims.append(ssim_avg)
        
        if args.save_result:
            save_result(np.hstack((psnrs, ssims)),path=os.path.join(args.result_dir,set_cur,'sigma_'+str(args.sigma)+'_results.txt'))
            save_result(np.hstack((snrs_in, psnrs_in, ssims_in)),path=os.path.join(args.result_dir,set_cur,'sigma_'+str(args.sigma)+'_results_input.txt'))

        log('Datset: {0:10s} \n  SNR_in = {1:2.2f} dB, PSNR_in = {2:2.2f} dB, SSIM_in = {3:1.4f}'.format(set_cur, snr_in_avg, psnr_in_avg, ssim_in_avg))
        log('PSNR = {0:2.2f} dB, SSIM = {1:1.4f}'.format(psnr_avg, ssim_avg))



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
